In [89]:
import pandas as pd
import numpy as np

In [90]:
df=pd.read_csv('/kaggle/working/train_prep_new.csv')

In [179]:
features=df.drop('Loan Sanction Amount (USD)',axis=1)
label=df['Loan Sanction Amount (USD)']
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler
xtrain,xtest,ytrain,ytest=train_test_split(features,label,random_state=42,test_size=0.1)

In [180]:
sc=MinMaxScaler()
xtrain=sc.fit_transform(xtrain)
xtest=sc.transform(xtest)
sc2=MinMaxScaler()
ytrain=sc2.fit_transform(ytrain.values.reshape(-1,1))
ytest=sc2.transform(ytest.values.reshape(-1,1))

In [181]:
ytrain=ytrain.reshape(-1,)
ytrain

array([0.33867121, 0.        , 0.27359133, ..., 0.16259131, 0.10988648,
       0.14062159])

In [182]:
ytest=ytest.reshape(-1,)

In [183]:
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
param={'n_estimators':np.arange(70,150,10),
       'max_features':['log2','sqrt','auto'],'max_depth':[2,3,4,5]}
rcv=RandomizedSearchCV(estimator=GradientBoostingRegressor(),param_distributions=param,scoring='r2')
rcv.fit(xtrain,ytrain)

RandomizedSearchCV(estimator=GradientBoostingRegressor(),
                   param_distributions={'max_depth': [2, 3, 4, 5],
                                        'max_features': ['log2', 'sqrt',
                                                         'auto'],
                                        'n_estimators': array([ 70,  80,  90, 100, 110, 120, 130, 140])},
                   scoring='r2')

In [184]:
rcv.best_params_

{'n_estimators': 80, 'max_features': 'auto', 'max_depth': 5}

In [185]:
rf=GradientBoostingRegressor(n_estimators=80,max_features='auto',max_depth=5)
rf.fit(xtrain,ytrain)

GradientBoostingRegressor(max_depth=5, max_features='auto', n_estimators=80)

In [186]:
pred=rf.predict(xtest)

In [187]:
from sklearn.metrics import r2_score
print(r2_score(ytest,pred))

0.7843649690996468


In [188]:
train=df.drop('Loan Sanction Amount (USD)',axis=1)
sc2=MinMaxScaler()
label=df['Loan Sanction Amount (USD)']
label=sc2.fit_transform(label.values.reshape(-1,1))
label=label.reshape(-1,)
sc=MinMaxScaler()
train=sc.fit_transform(train)
rf=GradientBoostingRegressor(n_estimators=80,max_features='auto',max_depth=5) # yielded r2 of 81.10054 --> RandomForest
rf.fit(train,label) # 81.13340 GBR(est=80,features=auto,depth=5)

GradientBoostingRegressor(max_depth=5, max_features='auto', n_estimators=80)

In [189]:
# test---
test=pd.read_csv('/kaggle/working/test_prep_new.csv')
test=test[df.columns[:-1]]

In [190]:
test=sc.transform(test)

In [191]:
pred=rf.predict(test)

In [192]:
pred

array([ 0.18465911,  0.13718126,  0.00965748, ..., -0.00504292,
        0.14853998,  0.14479577])

In [193]:
pred=sc2.inverse_transform(pred.reshape(-1,1))

In [194]:
submit=pd.read_csv('/kaggle/input/cipla-input/test.csv')

submit=submit['Customer ID']
submit=pd.DataFrame(submit)

submit['Loan Sanction Amount (USD)']=pred

submit

,Customer ID,Loan Sanction Amount (USD)
0,C-26247,88988.577908
1,C-35067,66108.653172
2,C-34590,4654.012641
3,C-16668,46648.473217
4,C-12196,64833.075443
...,...,...
19995,C-9076,92883.401312
19996,C-17587,28926.555874
19997,C-46479,-2430.220417
19998,C-3099,71582.504579


In [195]:
submit['Loan Sanction Amount (USD)'][submit['Loan Sanction Amount (USD)']<0]=0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [196]:
submit.to_csv('submit2.csv',index=False)